### Applied Data Science Captsone Project: Week 3

# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

This notebook contains code I have written to:
* Scrape data from Wikipedia regarding different neighbourhoods in Toronto, Canada.
* Transform the data as required in the assignment instructions.

### Install / load required packages

In [78]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### Scrape data

In [2]:
html = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(html)

# Create a BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser')

In [3]:
# Have a look at the object...
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":906439794,"wgRevisionId":906439794,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June",

In [4]:
# Find the table in the object that has class 'wikitable sortable'
My_table = soup.find('table',{'class':'wikitable sortable'})
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [11]:
# Create empty dataframe to store scraped data and a second dataframe to collect one table row at a time and append to main table
d = {'Postcode': [''], 'Borough': [''], 'Neighbourhood': ['']}
df = pd.DataFrame(data = d)
df_append = pd.DataFrame(data = d)

# Loop through all rows in the table and extract data
for row in My_table.find_all('tr'):
    b_process = False
    column_marker = 0
    row_data = row.find_all('td')
    for data in row_data:
        value = data.get_text().strip()
        df_append.iat[0,column_marker] = value
        if column_marker == 1: # Check to see if Borough is not assigned and don't process if that is the case
            if value != 'Not assigned':
                b_process = True
        column_marker += 1
    if b_process == True:
        df = df.append(df_append, ignore_index=True)
    
# Drop the first row (blank row included initially to allow for index to exist)
df.drop(df.index[0], inplace=True)
df.reset_index(drop=True, inplace=True)
#View dataframe
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Transform / clean the data

We have scraped the website data into a dataframe but there are further transformations required to achieve the desired result.

Let's group identical postcodes and concatenate the Neighbourhoods that share them. Note that there are some Boroughs that span multiple Postcodes (e.g. North York), and I am grouping by Postcode AND Borough to maintain as much information as possible.

In [12]:
df['Neighbourhood'] = df[['Postcode','Borough','Neighbourhood']].groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ', '.join(x))
df = df[['Postcode','Borough','Neighbourhood']].drop_duplicates()
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Not assigned


Now let's make sure any unassigned Neighbourhoods take their name from their Borough instead.

In [13]:
df.loc[df['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = df['Borough']
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park


### Check dimensions

Now we are done we can check the shape (dimensions) of the dataframe:

In [14]:
df.shape

(103, 3)

### Get latitudes and longitudes

Now we would like to get the latitudes and longitudes of each postcode. To do this I will use the gcsv file provided as geolocator (and geopy which I also tried) could not find the co-ordinates.

In [15]:
# Read csv using pandas
df_coords = pd.read_csv('Geospatial_Coordinates.csv')
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Add latitude and longitude column to dataframe 'df' using data from the csv file.

In [17]:
df2 = pd.merge(df, df_coords, left_on='Postcode', right_on='Postal Code', how='left')
df2.drop('Postal Code', axis=1, inplace=True)
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


### Perform clustering analysis

Now I will repeate the analysis we did for New York but with these Toronto neighourhoods.

Let's start with just visualising the neighbourhoods on a Folium map:

In [25]:
# create map of Toronto using latitude and longitude values
tor_lat = 43.6532
tor_long = -79.3832
map_tor = folium.Map(location=[tor_lat, tor_long], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

Now let's define some Foursquare API credentials...

In [29]:
CLIENT_ID = 'G0S3YO5MFLPCT2TQUOUEGQV5SUH3ELFMHFOV2OKC4YLUQL52' # your Foursquare ID
CLIENT_SECRET = 'XDPQ0XCEBNII0AS1Z21LLGHQJGCRALX2Q55GHSY5PKCGND0B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G0S3YO5MFLPCT2TQUOUEGQV5SUH3ELFMHFOV2OKC4YLUQL52
CLIENT_SECRET:XDPQ0XCEBNII0AS1Z21LLGHQJGCRALX2Q55GHSY5PKCGND0B


And our function that will use the Foursquare API to explore the area around each neighbourhood for venues and collect that information in a new dataframe...

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    
    return(nearby_venues)

Let's run the function on our dataframe of neighbourhoods...

In [36]:
toronto_venues = getNearbyVenues(names=df2['Neighbourhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

And here is the output:

In [112]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


Now let's use this data for some clustering!

Firstly I will group the venue data to see how many venues have been found for each neighbourhood.

Oh, this is less than when we did it for New York - that's ok, but perhaps later I will review clusters by looking at the top 5 venue types rather than the top 10. I will also remove any neighbourhoods with less than 5 venues from the analysis as they will likely be 'far away' from others in clustering terms, simply because they have no venues.

In [132]:
toronto_venues_sum = toronto_venues.groupby('Neighbourhood').count()
toronto_venues_sum

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,30,30,30,30,30,30


Let's also see how many types of venue were found in total...

In [114]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 231 uniques categories.


To create a dataset on which k-means clustering can be performed I will pivot the data using a one hot encoding technique. Then I will group by neighbourhood, taking the mean of the one hot encoding as the aggregation metric. This effectively creates a dataframe telling us what proportion of the venues found nearby are of each type - and this is effectively normalised as a result.

In [115]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

As I mentioned above I will remove any neighbourhoods from the analysis that have less than 5 venues in total, as follows:

In [158]:
toronto_venues_sum_5plus = toronto_venues_sum[toronto_venues_sum['Venue'] > 4]
toronto_venues_sum_5plus = toronto_venues_sum_5plus.reset_index()

toronto_onehot = pd.merge(toronto_venues_sum_5plus['Neighbourhood'], toronto_onehot, left_on='Neighbourhood', right_on='Neighbourhood', how='left')

In [161]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.033333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.000000,0.0,0.0,0.033333,0.000000,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.066667,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,

To make this a little more readable on a neighbourhood-by-neighbourhood basis I can define the following function that selects the top venue categories, and use it in the following code to summarise the neighborhoods:

In [162]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [163]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Café,Hotel,Coffee Shop,Asian Restaurant
1,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Pizza Place,Coffee Shop,Sandwich Place
2,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Coffee Shop,Pharmacy,Pool
3,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Fried Chicken Joint,Pizza Place,Middle Eastern Restaurant,Deli / Bodega
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Juice Bar,Thai Restaurant,Comfort Food Restaurant


Now, on to the clustering. I have used k-means clustering with k=5 on this occassion.

In [164]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 4, 0, 3, 2, 3, 2, 2, 3, 3])

Combining the clustering labels determined by the model with our top venue categories by neighborhood dataframe, we have something that will allow us to get an understanding of what each cluster looks like.

In [165]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [196]:
df_temp = toronto_venues.groupby('Neighbourhood').count().reset_index()
df_temp.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide, King, Richmond",30,30,30,30,30,30
1,Agincourt,4,4,4,4,4,4
2,"Agincourt North, L'Amoreaux East, Milliken, St...",3,3,3,3,3,3
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",11,11,11,11,11,11
4,"Alderwood, Long Branch",10,10,10,10,10,10


In [198]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = pd.merge(df2, neighbourhoods_venues_sorted, left_on='Neighbourhood', right_on='Neighbourhood', how='right')

# let's also capture how many venues were actually found in the first place to get some context - should be at least 5 in all cases
df_temp = toronto_venues.groupby('Neighbourhood').count().reset_index()
df_temp = df_temp.rename(columns = {'Venue Category' : 'Venue Count'})

toronto_merged = pd.merge(toronto_merged, df_temp[['Neighbourhood','Venue Count']], left_on='Neighbourhood', right_on='Neighbourhood', how='left')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Venue Count
0,M4A,North York,Victoria Village,43.725882,-79.315572,3,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,French Restaurant,6
1,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,3,Coffee Shop,Bakery,Park,Gym / Fitness Center,Mexican Restaurant,30
2,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,3,Clothing Store,Women's Store,Accessories Store,Vietnamese Restaurant,Miscellaneous Shop,11
3,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,3,Coffee Shop,Diner,Park,Gym,Yoga Studio,30
4,M3B,North York,Don Mills North,43.745906,-79.352188,2,Gym / Fitness Center,Caribbean Restaurant,Basketball Court,Baseball Field,Japanese Restaurant,6


In [199]:
# create map
map_clusters = folium.Map(location=[tor_lat, tor_long], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### What does cluster 0 look like? Pizza places feature heavily.

In [200]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0,toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Venue Count
5,"Woodbine Gardens, Parkview Hill",0,Pizza Place,Fast Food Restaurant,Pharmacy,Bank,Breakfast Spot,11
12,"Guildwood, Morningside, West Hill",0,Pizza Place,Breakfast Spot,Electronics Store,Medical Center,Mexican Restaurant,7
53,"Clarks Corners, Sullivan, Tam O'Shanter",0,Pizza Place,Chinese Restaurant,Shopping Mall,Fast Food Restaurant,Italian Restaurant,13
62,"Alderwood, Long Branch",0,Pizza Place,Athletics & Sports,Coffee Shop,Pharmacy,Pool,10


### What does cluster 1 look like? Parks are most common - and few venues overall per neighbourhood.

In [201]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Venue Count
14,Caledonia-Fairbanks,1,Park,Fast Food Restaurant,Women's Store,Market,Dessert Shop,5
40,York Mills West,1,Park,Convenience Store,Bar,Bank,Yoga Studio,5
60,Rosedale,1,Park,Playground,Building,Trail,Yoga Studio,5


### What does cluster 2 look like? Lot's of cafes and coffee shops - generally densely populated with venues.

In [202]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Venue Count
4,Don Mills North,2,Gym / Fitness Center,Caribbean Restaurant,Basketball Court,Baseball Field,Japanese Restaurant,6
6,"Ryerson, Garden District",2,Café,Coffee Shop,Clothing Store,Beer Bar,Plaza,30
10,St. James Town,2,Coffee Shop,Gastropub,Restaurant,Hotel,Japanese Restaurant,30
11,"Bloordale Gardens, Eringate, Markland Wood, Ol...",2,Liquor Store,Beer Store,Pizza Place,Coffee Shop,Convenience Store,8
16,Central Bay Street,2,Coffee Shop,Bubble Tea Shop,Spa,Italian Restaurant,Café,30
17,Christie,2,Grocery Store,Café,Park,Bank,Baby Store,17
21,"Adelaide, King, Richmond",2,Steakhouse,Café,Hotel,Coffee Shop,Asian Restaurant,30
25,"Harbourfront East, Toronto Islands, Union Station",2,Hotel,Park,Plaza,Café,Event Space,30
29,"Design Exchange, Toronto Dominion Centre",2,Coffee Shop,Restaurant,Café,Deli / Bodega,Bakery,30
30,"Brockton, Exhibition Place, Parkdale Village",2,Café,Breakfast Spot,Coffee Shop,Climbing Gym,Italian Restaurant,21


### What does cluster 3 look like? More of a mixed bag, with a variety of eateries.

In [203]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3,toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Venue Count
0,Victoria Village,3,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,French Restaurant,6
1,"Harbourfront, Regent Park",3,Coffee Shop,Bakery,Park,Gym / Fitness Center,Mexican Restaurant,30
2,"Lawrence Heights, Lawrence Manor",3,Clothing Store,Women's Store,Accessories Store,Vietnamese Restaurant,Miscellaneous Shop,11
3,Queen's Park,3,Coffee Shop,Diner,Park,Gym,Yoga Studio,30
7,Glencairn,3,Pizza Place,Pub,Japanese Restaurant,Park,Asian Restaurant,5
8,"Flemingdon Park, Don Mills South",3,Gym,Coffee Shop,Beer Store,Asian Restaurant,Restaurant,21
9,Woodbine Heights,3,Pharmacy,Cosmetics Shop,Skating Rink,Beer Store,Curling Ice,10
13,Berczy Park,3,Cocktail Bar,Seafood Restaurant,Coffee Shop,Farmers Market,Beer Bar,30
15,Leaside,3,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Burger Joint,Brewery,30
18,Cedarbrae,3,Hakka Restaurant,Fried Chicken Joint,Lounge,Thai Restaurant,Bakery,8


### What does cluster 4 look like? Still a variety of eateries, but perhaps more discount/department stores, and overall fewer venues.

In [204]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4,toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Venue Count
27,"East Birchmount Park, Ionview, Kennedy Park",4,Coffee Shop,Convenience Store,Chinese Restaurant,Discount Store,Department Store,5
43,Westmount,4,Pizza Place,Coffee Shop,Middle Eastern Restaurant,Chinese Restaurant,Discount Store,8
45,Willowdale West,4,Pharmacy,Coffee Shop,Discount Store,Pizza Place,Grocery Store,5
58,"Albion Gardens, Beaumond Heights, Humbergate, ...",4,Grocery Store,Fried Chicken Joint,Pizza Place,Coffee Shop,Sandwich Place,11
59,L'Amoreaux West,4,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Gym Pool,Pizza Place,12
